## Sentiment analysis of the lyrics

We will use the same dataframe use to build the network. It has merged songs duplications.

In [ ]:
#load de data from .pkl file
import pickle
import numpy as np
import pandas as pd

import os
import json
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
import string

from collections import defaultdict

In [5]:
#load de data from .pkl file
loaded_df = pd.read_pickle("ts_data.pkl")


In [6]:
loaded_df.head()

,track_name,track_musical_genre,track_type,duration_ms,feature,track_videoclip,videoclip_views,spotify_streams,spotify_global_peak,album,...,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,cleaned_lyrics
0,"""Slut!"" (Taylor's Version) (From The Vault)",Synth Pop,Single,180381,No,No,0,239579759,2,"[1989 (Taylor's Version) [Deluxe], 1989 (Taylo...",...,"[0.309, 0.345]","[0.591, 0.629]","[0.412, 0.413]","[0.000199, 7.62e-05]","[0.0608, 0.0603]","[-13.27, -13.178]","[0.0863, 0.055]","[155.875, 77.983]","[0.328, 0.306]",flamingo pink sunrise boulevard clink clink yo...
1,...Ready For It?,Electropop,Single,208186,No,Yes,350.707.005,695895392,3,reputation,...,0.0527,0.613,0.764,0.0,0.197,-6.509,0.136,160.015,0.417,knew killer first time saw wondered many girl ...
2,A Perfectly Good Heart,Country Pop,B-Side,220146,No,No,0,25204096,0,Taylor Swift,...,0.00349,0.483,0.751,0.0,0.128,-5.726,0.0365,156.092,0.268,would wanna break perfectly good heart would w...
3,A Place in this World,Country Pop,B-Side,"[202080, 199200]",No,No,0,40680546,0,[reputation Stadium Tour Surprise Song Playlis...,...,"[0.0577, 0.051]","[0.573, 0.576]","[0.767, 0.777]",0.0,"[0.327, 0.32]","[-2.929, -2.881]","[0.0323, 0.0324]","[114.984, 115.028]","[0.438, 0.428]",dont know want dont ask cause im still trying ...
4,Afterglow,Pop,B-Side,223293,No,No,0,368409481,38,Lover,...,0.13,0.756,0.449,0.0,0.114,-8.746,0.0344,111.011,0.399,blew thing proportion youre blue put jail some...


In [7]:
len(loaded_df)

235

In [8]:
loaded_df.columns

Index(['track_name', 'track_musical_genre', 'track_type', 'duration_ms',
       'feature', 'track_videoclip', 'videoclip_views', 'spotify_streams',
       'spotify_global_peak', 'album', 'track_number', 'album_musical_genre',
       'album_type', 'release_date', 'album_physical_sales', 'track_lyrics',
       'track_theme', 'uri', 'acousticness', 'danceability', 'energy',
       'instrumentalness', 'liveness', 'loudness', 'speechiness', 'tempo',
       'valence', 'cleaned_lyrics'],
      dtype='object')

In [9]:
def sentiment_dictionary(LabMT = 'data/Data_Set_S1.txt'):
    """
    This function loads the LabMT wordlist and creates a dictionary of words and their happiness values.
    labMT: str, the path to the LabMT wordlist
    """
    #load the LabMT wordlist from Data_Set_S1.txt
    with open(LabMT, 'r') as f:
        lines = f.readlines()
    
    #remove the first foour lines 
    lines = lines[4:]

    #create a dictionary of words and their happiness values
    word_dict = defaultdict(float)
    for line in lines:
        line = line.split('\t')
        word = line[0]
        happiness = float(line[2])
        #print(word, happiness)
        word_dict[word] = happiness

    return word_dict

In [10]:
def lyrics_to_tokens(lyrics):
    """
    This function takes a string of lyrics and returns a list of tokens.
    lyrics: str, a string of lyrics    
    """
    #convert to lower case
    lyrics = lyrics.lower()
    #remove stopwords
    stop_words = set(stopwords.words('english'))
    #remove punctuation
    lyrics = lyrics.translate(str.maketrans('', '', string.punctuation))
    #tokenize the lyrics
    tokens = word_tokenize(lyrics)

    tokens = [word for word in tokens if word not in stop_words]
    #lemmatize the tokens
    #lemmatizer = WordNetLemmatizer() We do not lemmatize the words because the LabMT wordlist is not lemmatized
    return tokens

In [11]:
#function to calculate sentiment given a list of tokens based on the LabMT wordlist dictionary
"""def sentiment(lyrics, word_dict):
    
    #This function calculates the sentiment of a list of tokens based on the LabMT wordlist dictionary.
    #tokens: list, a list of tokens
    #word_dict: dict, a dictionary of words and their happiness values
    
    tokens = lyrics_to_tokens(lyrics)
    #calculate the frequency of each word in the tokens list
    freq = defaultdict(int)
    for token in tokens:
        freq[token] += 1

    #calculate the happiness score of the tokens list
    score = 0
    count = 0
    for word in freq:
        if word in word_dict:
            score += freq[word] * word_dict[word]
            count += freq[word]
            sentiment = score / count

    return sentiment"""

'def sentiment(lyrics, word_dict):\n    \n    #This function calculates the sentiment of a list of tokens based on the LabMT wordlist dictionary.\n    #tokens: list, a list of tokens\n    #word_dict: dict, a dictionary of words and their happiness values\n    \n    tokens = lyrics_to_tokens(lyrics)\n    #calculate the frequency of each word in the tokens list\n    freq = defaultdict(int)\n    for token in tokens:\n        freq[token] += 1\n\n    #calculate the happiness score of the tokens list\n    score = 0\n    count = 0\n    for word in freq:\n        if word in word_dict:\n            score += freq[word] * word_dict[word]\n            count += freq[word]\n            sentiment = score / count\n\n    return sentiment'

In [12]:
#function to calculate sentiment given a list of tokens based on the LabMT wordlist dictionary
def sentiment_with_words(lyrics, word_dict):
    """
    This function calculates the sentiment of a list of tokens based on the LabMT wordlist dictionary. 
    It also returns the words that are in the LabMT wordlist, their frequency and scores
    tokens: list, a list of tokens
    word_dict: dict, a dictionary of words and their happiness values
    """
    tokens = lyrics_to_tokens(lyrics)
    #calculate the frequency of each word in the tokens list
    freq = defaultdict(int)
    for token in tokens:
        freq[token] += 1

    word_list = []
    #calculate the happiness score of the tokens list
    score = 0
    count = 0
    for word in freq:
        if word in word_dict:
            word_list.append((word, freq[word], word_dict[word]))
            score += freq[word] * word_dict[word]
            count += freq[word]
            sentiment = score / count
    return sentiment, word_list


In [13]:
#function to calculate sentiment of each track in the dataframe
"""def calculate_sentiments(df):
    
    #This function calculates the sentiment of each track in the dataframe.
    #df: pandas dataframe, a dataframe with a column of track lyrics
    
    word_dict = sentiment_dictionary()
    sentiments_dict = defaultdict(list)
    for index, row in df.iterrows():
        lyrics = row['track_lyrics']
        if type(lyrics) == list:
            for lyric in lyrics:
                sentiment_score = sentiment(lyric, word_dict)
                sentiments_dict[row['track_name']].append(sentiment_score)
        else:
            sentiment_score = sentiment(lyrics, word_dict)
            sentiments_dict[row['track_name']].append(sentiment_score)

    return sentiments_dict"""

"def calculate_sentiments(df):\n    \n    #This function calculates the sentiment of each track in the dataframe.\n    #df: pandas dataframe, a dataframe with a column of track lyrics\n    \n    word_dict = sentiment_dictionary()\n    sentiments_dict = defaultdict(list)\n    for index, row in df.iterrows():\n        lyrics = row['track_lyrics']\n        if type(lyrics) == list:\n            for lyric in lyrics:\n                sentiment_score = sentiment(lyric, word_dict)\n                sentiments_dict[row['track_name']].append(sentiment_score)\n        else:\n            sentiment_score = sentiment(lyrics, word_dict)\n            sentiments_dict[row['track_name']].append(sentiment_score)\n\n    return sentiments_dict"

In [14]:
def calculate_sentiments_with_details(df):
    """
    This function calculates the sentiment of each track in the dataframe and returns the happiest and saddest words.
    df: pandas dataframe, a dataframe with a column of track lyrics
    """
    # add the three happiest words and three saddest words to the sentiments_dict with their scores and frequencies
    word_dict = sentiment_dictionary()
    sentiments_dict = defaultdict(dict)
    for index, row in df.iterrows():
        lyrics = row['track_lyrics']
        if type(lyrics) == list:
            for lyric in lyrics:
                sentiment_score, word_list = sentiment_with_words(lyric, word_dict)
                sentiments_dict[row['track_name']]['sentiment'] = sentiment_score
                #order the word_list by score
                word_list = sorted(word_list, key=lambda x: x[2])
                sentiments_dict[row['track_name']]['saddest words'] = word_list[:3]
                sentiments_dict[row['track_name']]['happiest words'] = word_list[-3:]
        else:
            sentiment_score, word_list = sentiment_with_words(lyrics, word_dict)
            sentiments_dict[row['track_name']]['sentiment'] = sentiment_score
            #order the word_list by score
            word_list = sorted(word_list, key=lambda x: x[2])
            sentiments_dict[row['track_name']]['saddest words'] = word_list[:3]
            sentiments_dict[row['track_name']]['happiest words'] = word_list[-3:]

    return sentiments_dict

In [15]:
song_sentiments_with_details = calculate_sentiments_with_details(loaded_df)
song_sentiments_with_details

defaultdict(dict,
            {'"Slut!" (Taylor\'s Version) (From The Vault)': {'sentiment': 5.483416149068322,
              'saddest words': [('crime', 1, 2.2),
               ('ill', 1, 2.42),
               ('mistake', 1, 2.78)],
              'happiest words': [('rose', 1, 7.32),
               ('pretty', 1, 7.32),
               ('love', 7, 8.42)]},
             '...Ready For It?': {'sentiment': 5.727163461538459,
              'saddest words': [('failure', 1, 2.06),
               ('killer', 1, 2.42),
               ('stealing', 1, 2.46)],
              'happiest words': [('dreams', 8, 7.44),
               ('loved', 1, 7.96),
               ('love', 1, 8.42)]},
             'A Perfectly Good Heart': {'sentiment': 5.503359999999999,
              'saddest words': [('tear', 4, 3.1),
               ('scar', 4, 3.28),
               ('cant', 1, 3.48)],
              'happiest words': [('heart', 7, 7.22),
               ('perfectly', 7, 7.28),
               ('love', 4, 8.42)]},
   

In [16]:
#calculate the stats of all the sentiments
sentiments = [value['sentiment'] for value in song_sentiments_with_details.values()]
sentiments = np.array(sentiments)

print('Mean sentiment:', np.mean(sentiments))
print('Median sentiment:', np.median(sentiments))
print('Standard deviation sentiment:', np.std(sentiments))
print('Minimum sentiment:', np.min(sentiments))
print('Maximum sentiment:', np.max(sentiments))

Mean sentiment: 5.511524910721485
Median sentiment: 5.526406250000001
Standard deviation sentiment: 0.24273196012242804
Minimum sentiment: 4.608599999999999
Maximum sentiment: 6.236


In [ ]:
#calculate the stats of the happiest words. We will use the happiest words from all the songs taking into account that they may be repeated
happiest_words = []
for value in song_sentiments_with_details.values():
    happiest_words.extend([word[0] for word in value['happiest words']])
happiest_words = np.array(happiest_words)

In [ ]:
#calculate the percentage of sad words and happy words in each song. We have to set a threshold to separate sad and happy words